## ORGANS: Liver Import + Export Flow

This notebook contains analysis of the import and export flow of livers between states.

---

### Initialize environment

In [2]:
library(tidyverse)

In [3]:
root_path     <- here::here()
data_path     <- paste0(root_path, "/01_data")
analysis_path <- paste0(root_path, "/02_analysis")

In [4]:
setwd(data_path)

transplant <- readRDS("clean/transplant_clean.rds")

setwd(analysis_path)

---

### Calculate number of livers coming into and going out of each state

In [5]:
txByYearDonCanState <- transplant %>%
  filter(REC_TX_DT_YEAR %in% 2018:2021) %>%
  # excluding 1701 transplants with missing donor information
  tidylog::filter(!is.na(DON_HOSPITAL_STATE)) %>%
  # excluding 3 transplants with missing candidate information
  tidylog::filter(!is.na(CAN_HOSPITAL_STATE)) %>%
  count(REC_TX_DT_YEAR, DON_HOSPITAL_STATE, CAN_HOSPITAL_STATE, name = "N_TX") %>%
  group_by(REC_TX_DT_YEAR, DON_HOSPITAL_STATE) %>%
  mutate(PCT_TX_DON_STATE = N_TX / sum(N_TX)) %>%
  ungroup() %>%
  group_by(REC_TX_DT_YEAR, CAN_HOSPITAL_STATE) %>%
  mutate(PCT_TX_CAN_STATE = N_TX / sum(N_TX)) %>%
  ungroup() %>%
  arrange(REC_TX_DT_YEAR, DON_HOSPITAL_STATE, -N_TX)

write.csv(x = txByYearDonCanState,
          file = "output/tables/table3-tx_by_year_donstate_canstate.csv",
          row.names = F)

txByYearDonCanState %>% head()

filter: removed 1,701 rows (5%), 31,468 rows remaining

filter: removed 3 rows (<1%), 31,465 rows remaining



REC_TX_DT_YEAR,DON_HOSPITAL_STATE,CAN_HOSPITAL_STATE,N_TX,PCT_TX_DON_STATE,PCT_TX_CAN_STATE
<dbl>,<chr>,<chr>,<int>,<dbl>,<dbl>
2018,AK,WA,15,0.78947368,0.106382979
2018,AK,OR,3,0.15789474,0.036585366
2018,AK,AZ,1,0.05263158,0.003571429
2018,AL,AL,85,0.68000000,0.714285714
2018,AL,FL,17,0.13600000,0.032882012
2018,AL,GA,8,0.06400000,0.030534351


In [6]:
importExportInternalTxByYearState <- bind_rows(
  txByYearDonCanState %>%
    filter(DON_HOSPITAL_STATE == CAN_HOSPITAL_STATE) %>%
    select(YEAR = REC_TX_DT_YEAR, STATE = DON_HOSPITAL_STATE, N = N_TX) %>%
    mutate(TYPE = "N_INTERNAL_LIVERS"),
  txByYearDonCanState %>%
    filter(DON_HOSPITAL_STATE != CAN_HOSPITAL_STATE) %>%
    count(YEAR = REC_TX_DT_YEAR, STATE = DON_HOSPITAL_STATE, wt = N_TX, name = "N") %>%
    mutate(TYPE = "N_EXPORTED_LIVERS"),
  txByYearDonCanState %>%
    filter(DON_HOSPITAL_STATE != CAN_HOSPITAL_STATE) %>%
    count(YEAR = REC_TX_DT_YEAR, STATE = CAN_HOSPITAL_STATE, wt = N_TX, name = "N") %>%
    mutate(TYPE = "N_IMPORTED_LIVERS")) %>%
  pivot_wider(id_cols = c("YEAR", "STATE"), names_from = TYPE, values_from = N) %>%
  mutate(across(matches("^N_"), replace_na, 0)) %>%
  mutate(MAJORITY_EXPORTER = N_EXPORTED_LIVERS > N_INTERNAL_LIVERS,
         MAJORITY_IMPORTER = N_IMPORTED_LIVERS > N_INTERNAL_LIVERS) %>%
  arrange(YEAR, STATE)

write.csv(x = importExportInternalTxByYearState,
          file = "output/tables/table3-tx_import_export_internal_by_year_state.csv",
          row.names = F)

importExportInternalTxByYearState %>% head()

YEAR,STATE,N_INTERNAL_LIVERS,N_EXPORTED_LIVERS,N_IMPORTED_LIVERS,MAJORITY_EXPORTER,MAJORITY_IMPORTER
<dbl>,<chr>,<int>,<int>,<int>,<lgl>,<lgl>
2018,AK,0,19,0,TRUE,FALSE
2018,AL,85,40,34,FALSE,FALSE
2018,AR,27,23,10,FALSE,FALSE
2018,AZ,112,60,168,FALSE,TRUE
2018,CA,566,115,219,FALSE,FALSE
2018,CO,61,24,46,FALSE,FALSE


In [7]:
importExportInternalTxByYearState %>%
  group_by(YEAR) %>%
  summarise(across(matches("^N_"), sum, na.rm = T)) %>%
  mutate(PCT_IMPORTED_LIVERS = N_IMPORTED_LIVERS / (N_INTERNAL_LIVERS + N_IMPORTED_LIVERS),
         PCT_EXPORTED_LIVERS = N_EXPORTED_LIVERS / (N_INTERNAL_LIVERS + N_EXPORTED_LIVERS))

YEAR,N_INTERNAL_LIVERS,N_EXPORTED_LIVERS,N_IMPORTED_LIVERS,PCT_IMPORTED_LIVERS,PCT_EXPORTED_LIVERS
<dbl>,<int>,<int>,<int>,<dbl>,<dbl>
2018,5014,2334,2334,0.3176375,0.3176375
2019,5378,2522,2522,0.3192405,0.3192405
2020,3856,4122,4122,0.5166708,0.5166708
2021,3717,4522,4522,0.5488530,0.5488530


In [8]:
importExportShift <- importExportInternalTxByYearState %>%
  mutate(PCT_IMPORT = N_IMPORTED_LIVERS / (N_INTERNAL_LIVERS + N_IMPORTED_LIVERS),
         PCT_EXPORT = N_EXPORTED_LIVERS / (N_INTERNAL_LIVERS + N_EXPORTED_LIVERS)) %>%
  filter(YEAR %in% c(2019, 2021)) %>%
  pivot_wider(id_cols = STATE, names_from = YEAR, values_from = matches("PCT_")) %>%
  mutate(PCT_IMPORT_SHIFT = PCT_IMPORT_2021 - PCT_IMPORT_2019, .after = PCT_IMPORT_2021) %>%
  mutate(PCT_EXPORT_SHIFT = PCT_EXPORT_2021 - PCT_EXPORT_2019, .after = PCT_EXPORT_2021)
 

print("Summary of PCT_IMPORT_SHIFT")
summary(importExportShift$PCT_IMPORT_SHIFT)

importExportShift %>%
  filter(between(PCT_IMPORT_SHIFT, 0.24, 0.25))

print("Summary of PCT_EXPORT_SHIFT")
summary(importExportShift$PCT_EXPORT_SHIFT)

importExportShift %>%
  filter(between(PCT_EXPORT_SHIFT, 0.185, 0.205))

importExportShift

[1] "Summary of PCT_IMPORT_SHIFT"


    Min.  1st Qu.   Median     Mean  3rd Qu.     Max.     NA's 
-0.05195  0.11143  0.24066  0.24470  0.34964  0.62076       14 

STATE,PCT_IMPORT_2019,PCT_IMPORT_2021,PCT_IMPORT_SHIFT,PCT_EXPORT_2019,PCT_EXPORT_2021,PCT_EXPORT_SHIFT
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TN,0.3185841,0.5592417,0.2406576,0.4166667,0.7066246,0.2899579


[1] "Summary of PCT_EXPORT_SHIFT"


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
-0.2707  0.0000  0.1959  0.1811  0.3393  0.4798       1 

STATE,PCT_IMPORT_2019,PCT_IMPORT_2021,PCT_IMPORT_SHIFT,PCT_EXPORT_2019,PCT_EXPORT_2021,PCT_EXPORT_SHIFT
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CO,0.4365079,0.5447761,0.1082682,0.3106796,0.5000000,0.1893204
CT,0.6136364,0.8823529,0.2687166,0.7118644,0.9142857,0.2024213


STATE,PCT_IMPORT_2019,PCT_IMPORT_2021,PCT_IMPORT_SHIFT,PCT_EXPORT_2019,PCT_EXPORT_2021,PCT_EXPORT_SHIFT
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AK,NaN,NaN,NaN,1.00000000,1.0000000,0.00000000
AL,0.28125000,0.56944444,0.28819444,0.36986301,0.8154762,0.44561318
AR,0.32558140,0.64444444,0.31886305,0.50847458,0.7837838,0.27530921
AZ,0.63496933,0.66559486,0.03062553,0.39593909,0.4851485,0.08920943
CA,0.27972028,0.29502762,0.01530734,0.16486486,0.1026723,-0.06219257
CO,0.43650794,0.54477612,0.10826818,0.31067961,0.5000000,0.18932039
CT,0.61363636,0.88235294,0.26871658,0.71186441,0.9142857,0.20242131
DC,0.80000000,0.91743119,0.11743119,0.40625000,0.6538462,0.24759615
DE,NaN,NaN,NaN,1.00000000,1.0000000,0.00000000


In [9]:
majorityStatusByYear <- importExportInternalTxByYearState %>%
  group_by(YEAR) %>%
  summarise(across(matches("MAJORITY"), ~ sum(as.numeric(.x))))

write.csv(x = majorityStatusByYear,
          file = "output/tables/table3-majority_import_export_states_by_year.csv",
          row.names = F)

majorityStatusByYear

YEAR,MAJORITY_EXPORTER,MAJORITY_IMPORTER
<dbl>,<dbl>,<dbl>
2018,22,7
2019,22,9
2020,38,30
2021,42,31


---

### Re-run same calculations, but using UNOS 730-day periods

In [10]:
unos_periods <- list(pre  = as.Date(c("2018-02-03", "2020-02-03")),
                     post = as.Date(c("2020-02-04", "2022-02-03")))

In [11]:
txByYearDonCanState.v2 <- transplant %>%
  filter(between(REC_TX_DT_DATE, unos_periods$pre[1], unos_periods$post[2])) %>%
  tidylog::filter(!is.na(CAN_HOSPITAL_STATE), !is.na(DON_HOSPITAL_STATE)) %>%
  mutate(PERIOD = case_when(between(REC_TX_DT_DATE, unos_periods$pre[1], unos_periods$pre[2]) ~ "PRE",
                            between(REC_TX_DT_DATE, unos_periods$post[1], unos_periods$post[2]) ~ "POST")) %>%
  count(PERIOD, DON_HOSPITAL_STATE, CAN_HOSPITAL_STATE, name = "N_TX") %>%
  group_by(PERIOD, DON_HOSPITAL_STATE) %>%
  mutate(PCT_TX_DON_STATE = N_TX / sum(N_TX)) %>%
  ungroup() %>%
  group_by(PERIOD, CAN_HOSPITAL_STATE) %>%
  mutate(PCT_TX_CAN_STATE = N_TX / sum(N_TX)) %>%
  ungroup() %>%
  arrange(PERIOD, DON_HOSPITAL_STATE, -N_TX)

txByYearDonCanState.v2 %>% head()

filter: removed 1,722 rows (5%), 31,487 rows remaining



PERIOD,DON_HOSPITAL_STATE,CAN_HOSPITAL_STATE,N_TX,PCT_TX_DON_STATE,PCT_TX_CAN_STATE
<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>
POST,AK,WA,21,0.55263158,0.078066914
POST,AK,OR,12,0.31578947,0.074074074
POST,AK,CA,3,0.07894737,0.001645639
POST,AK,AZ,2,0.05263158,0.003333333
POST,AL,GA,65,0.20376176,0.125000000
POST,AL,AL,60,0.18808777,0.370370370


In [12]:
importExportInternalTxByYearState.v2 <- bind_rows(
  txByYearDonCanState.v2 %>%
    filter(DON_HOSPITAL_STATE == CAN_HOSPITAL_STATE) %>%
    select(PERIOD, STATE = DON_HOSPITAL_STATE, N = N_TX) %>%
    mutate(TYPE = "N_INTERNAL_LIVERS"),
  txByYearDonCanState.v2 %>%
    filter(DON_HOSPITAL_STATE != CAN_HOSPITAL_STATE) %>%
    count(PERIOD, STATE = DON_HOSPITAL_STATE, wt = N_TX, name = "N") %>%
    mutate(TYPE = "N_EXPORTED_LIVERS"),
  txByYearDonCanState.v2 %>%
    filter(DON_HOSPITAL_STATE != CAN_HOSPITAL_STATE) %>%
    count(PERIOD, STATE = CAN_HOSPITAL_STATE, wt = N_TX, name = "N") %>%
    mutate(TYPE = "N_IMPORTED_LIVERS")) %>%
  pivot_wider(id_cols = c("PERIOD", "STATE"), names_from = TYPE, values_from = N) %>%
  mutate(across(matches("^N_"), replace_na, 0)) %>%
  mutate(MAJORITY_EXPORTER = N_EXPORTED_LIVERS > N_INTERNAL_LIVERS,
         MAJORITY_IMPORTER = N_IMPORTED_LIVERS > N_INTERNAL_LIVERS) %>%
  arrange(STATE, PERIOD)

importExportInternalTxByYearState.v2 %>% head()

PERIOD,STATE,N_INTERNAL_LIVERS,N_EXPORTED_LIVERS,N_IMPORTED_LIVERS,MAJORITY_EXPORTER,MAJORITY_IMPORTER
<chr>,<chr>,<int>,<int>,<int>,<lgl>,<lgl>
POST,AK,0,38,0,TRUE,FALSE
PRE,AK,0,46,0,TRUE,FALSE
POST,AL,60,259,102,TRUE,TRUE
PRE,AL,177,95,68,FALSE,FALSE
POST,AR,27,97,50,TRUE,TRUE
PRE,AR,57,51,24,FALSE,FALSE


In [13]:
importExportInternalTxByYearState.v2 %>%
  group_by(PERIOD) %>%
  summarise(across(matches("^N_"), sum, na.rm = T)) %>%
  mutate(PCT_IMPORTED_LIVERS = N_IMPORTED_LIVERS / (N_INTERNAL_LIVERS + N_IMPORTED_LIVERS),
         PCT_EXPORTED_LIVERS = N_EXPORTED_LIVERS / (N_INTERNAL_LIVERS + N_EXPORTED_LIVERS))

PERIOD,N_INTERNAL_LIVERS,N_EXPORTED_LIVERS,N_IMPORTED_LIVERS,PCT_IMPORTED_LIVERS,PCT_EXPORTED_LIVERS
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
POST,7365,8803,8803,0.5444706,0.5444706
PRE,10447,4872,4872,0.3180364,0.3180364


Using the 730-day periods, we're getting effectively the same pre- and post-policy shifts in national share of livers imported/exported (shifting from 31.8% to 54.4% using 730-days; shifting from 31.9% to 54.9% using 2019 vs. 2021)

Seeing similar results in next cell: the median import shift by state was +26.7pp (as compared to +24pp above); the median export shift by state was +20.8pp (as compared to 19.6pp above).

In [14]:
importExportShift.v2 <- importExportInternalTxByYearState.v2 %>%
  mutate(PCT_IMPORT = N_IMPORTED_LIVERS / (N_INTERNAL_LIVERS + N_IMPORTED_LIVERS),
         PCT_EXPORT = N_EXPORTED_LIVERS / (N_INTERNAL_LIVERS + N_EXPORTED_LIVERS)) %>%
  pivot_wider(id_cols = STATE, names_from = PERIOD, values_from = matches("PCT_")) %>%
  mutate(PCT_IMPORT_SHIFT = PCT_IMPORT_POST - PCT_IMPORT_PRE, .after = PCT_IMPORT_PRE) %>%
  mutate(PCT_EXPORT_SHIFT = PCT_EXPORT_POST - PCT_EXPORT_PRE, .after = PCT_EXPORT_PRE)
 
print("Summary of PCT_IMPORT_SHIFT")
summary(importExportShift.v2$PCT_IMPORT_SHIFT)

print("Summary of PCT_EXPORT_SHIFT")
summary(importExportShift.v2$PCT_EXPORT_SHIFT)

importExportShift.v2

[1] "Summary of PCT_IMPORT_SHIFT"


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
-0.0522  0.1588  0.2670  0.2533  0.3552  0.6433      14 

[1] "Summary of PCT_EXPORT_SHIFT"


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
-0.2583  0.0000  0.2084  0.1877  0.3190  0.5598       1 

STATE,PCT_IMPORT_POST,PCT_IMPORT_PRE,PCT_IMPORT_SHIFT,PCT_EXPORT_POST,PCT_EXPORT_PRE,PCT_EXPORT_SHIFT
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AK,NaN,NaN,NaN,1.0000000,1.00000000,0.00000000
AL,0.62962963,0.27755102,0.35207861,0.8119122,0.34926471,0.46264752
AR,0.64935065,0.29629630,0.35305435,0.7822581,0.47222222,0.31003584
AZ,0.69500000,0.62418301,0.07081699,0.5390428,0.37837838,0.16066444
CA,0.30005485,0.27801592,0.02203893,0.1175657,0.16854725,-0.05098155
CO,0.46747967,0.41949153,0.04798815,0.4842520,0.29743590,0.18681607
CT,0.92929293,0.61458333,0.31470960,0.9457364,0.67543860,0.27029784
DC,0.92647059,0.76683938,0.15963121,0.6875000,0.32835821,0.35914179
DE,NaN,NaN,NaN,1.0000000,1.00000000,0.00000000


In [15]:
majorityStatusByYear.v2 <- importExportInternalTxByYearState.v2 %>%
  group_by(PERIOD) %>%
  summarise(across(matches("MAJORITY"), ~ sum(as.numeric(.x))))

majorityStatusByYear.v2

PERIOD,MAJORITY_EXPORTER,MAJORITY_IMPORTER
<chr>,<dbl>,<dbl>
POST,42,30
PRE,21,8


Again getting very similar results: a shift of 21 major exports in pre-policy period to 42 in post-policy period, as compared to 22 in FY2019 vs. 42 in FY2021 (see above)